Protocol for checking CDS offtargets:

1. Get the augmented library guides (guides + 4 PAMs) and bowtie align them to CDS
    1. This finds the appropriate NGG PAM guide variants
    1. Two things can be observed from the alignment output:
        1. For 0mm, which guide + PAM sequences have exact matches and their locations
            1. If a guide has more than 1 exact match with the same or different PAM, those are off targets.
        1. For 1mm, there are alignments that indicate 1 mismatch but in the N of the PAM region. I throw these out. Because these target the same region as the "original" guide + PAM with which they mismatch once. If they targeted another region with 1 mismatch, there would have been a correct guide + PAM variant in the previous step (0mm) which would have been recorded already.
        1. For 2mm, there are alignments that indicate 1 mismatch in the N of the PAM, and 1 mismatch somewhere else in the sequence. I throw these out. Because the correct variant and 1 mismatch would have been recorded in the previous step (1mm)
1. The number of exact matches of 0MM CDS must be the same as the library size.
    1. Actually, size is a bad measure. What if the same guide with different PAMs has matches?
1. Need to align augmented library to CDS, then add a column without PAMS. This column must have the same elements and size as the unaugmented library guides. The size of the alignment will be equal or more (w/ offtargets) than the library.

if there is an intron separation event, we can't see it in the protocol above. Need to check genome.

1. Align the augmented library to genome
    1. This finds the appropriate NGG PAM guide variants.
1. align augmented library to genome, then add a column without PAMS. This column must have the same elements and size as the unaugmented library guides. The size of the alignment will be:

1. Equal
    1. When either all guides have exactly 1 match (good!), or some guides have NO match (intron event) AND some have more than 1 match (offtarget event, double bad)
1. Fewer
    1. When some guides have no match (intron event)
1. More
    1. When some guides have more than 1 match (offtarget)

than the library size

In [9]:
import os
import numpy as np
import pandas as pd

def rc(string):
    s = ''
    for c in string:
        if c == 'A': s += 'T'
        elif c == 'C': s += 'G'
        elif c == 'G': s += 'C'
        elif c == 'T': s += 'A'
    return s[::-1]

def GO(species_name, track):
    if not os.path.exists(f'{species_name}/output/{track}/cds_targets'):
        os.makedirs(f'{species_name}/output/{track}/cds_targets')
    
    if not os.path.exists(f'{species_name}/output/{track}/genome_targets'):
        os.makedirs(f'{species_name}/output/{track}/genome_targets')

    library = pd.read_csv(f'{species_name}/{species_name}_{track}_library.csv')

    zero_mm_cds_offtargets = [0] * len(library)
    one_mm_cds_offtargets = [0] * len(library)
    two_mm_cds_offtargets = [0] * len(library)

    zero_mm_genome_offtargets = [0] * len(library)
    one_mm_genome_offtargets = [0] * len(library)
    two_mm_genome_offtargets = [0] * len(library)


    df_a7_0mm_cds = pd.read_csv(f'{species_name}/output/{track}/{species_name}_{track}_0mm_cds_alignment.sam', sep='\t',
                names=['query_name', 'strand', 'reference_name', 'mapping_position',
                       'aligned_seq', 'mapping_quality', 'idk', 'mismatch'])

    df_a7_0mm_cds.drop(columns=['idk', 'mapping_quality'], inplace=True)

    guide_w_pam = list()

    # Add a reverse complement column if strand is negative - otherwise write the same sequence
    for idx, row in df_a7_0mm_cds.iterrows():
        if row['strand'] == '-':
            guide_w_pam.append(rc(row['aligned_seq']))
        else:
            guide_w_pam.append(row['aligned_seq'])

    df_a7_0mm_cds['guide_w_pam'] = guide_w_pam
    df_a7_0mm_cds['guide'] = df_a7_0mm_cds['guide_w_pam'].str[:-3]

    df_a7_0mm_cds['mismatch'].fillna('Exact Match', inplace=True)
    df_a7_0mm_cds = df_a7_0mm_cds[~df_a7_0mm_cds['mismatch'].str.contains('20:')]
    df_a7_0mm_cds['num_mutations'] = df_a7_0mm_cds['mismatch'].apply(lambda x: len(x.split(',')))
    df_a7_0mm_cds.loc[df_a7_0mm_cds['mismatch'] == 'Exact Match', 'num_mutations'] = 0

    df_a7_0mm_cds['num_mutations'] = df_a7_0mm_cds['num_mutations'].astype(int)

    has_no_offtargets = (set(library['sequence']) == set(df_a7_0mm_cds['guide'])) and (len(library['sequence']) == len(df_a7_0mm_cds['guide']))

    if not has_no_offtargets:
        for idx, row in library.iterrows():
            seq = row['sequence']
            zero_mm_cds_offtargets[idx] += len(df_a7_0mm_cds[df_a7_0mm_cds['guide'] == seq]) - 1

    ## -------
    ## CDS 1MM
    ## -------
    df_a7_1mm_cds = pd.read_csv(f'{species_name}/output/{track}/{species_name}_{track}_1mm_cds_alignment.sam', sep='\t',
                names=['query_name', 'strand', 'reference_name', 'mapping_position',
                       'aligned_seq', 'mapping_quality', 'idk', 'mismatch'])

    df_a7_1mm_cds.drop(columns=['idk', 'mapping_quality'], inplace=True)

    guide_w_pam = list()

    for idx, row in df_a7_1mm_cds.iterrows():
        if row['strand'] == '-':
            guide_w_pam.append(rc(row['aligned_seq']))
        else:
            guide_w_pam.append(row['aligned_seq'])

    df_a7_1mm_cds['guide_w_pam'] = guide_w_pam
    df_a7_1mm_cds['guide'] = df_a7_1mm_cds['guide_w_pam'].str[:-3]

    # Create a new column called num_mutations and extract the mismatches from
    # the mismatch column.
    df_a7_1mm_cds['mismatch'].fillna('Exact Match', inplace=True)
    df_a7_1mm_cds = df_a7_1mm_cds[~df_a7_1mm_cds['mismatch'].str.contains('20:')]
    df_a7_1mm_cds['num_mutations'] = df_a7_1mm_cds['mismatch'].apply(lambda x: len(x.split(',')))
    df_a7_1mm_cds.loc[df_a7_1mm_cds['mismatch'] == 'Exact Match', 'num_mutations'] = 0

    df_a7_1mm_cds['num_mutations'] = df_a7_1mm_cds['num_mutations'].astype(int)
    
    has_no_offtargets = (set(library['sequence']) == set(df_a7_1mm_cds['guide'])) and (len(library['sequence']) == len(df_a7_1mm_cds['guide']))

    if not has_no_offtargets:
        for _, row in df_a7_1mm_cds[df_a7_1mm_cds['num_mutations'] > 0].iterrows():
            idx = library[library['sequence'] == row['guide']].index[0]
            one_mm_cds_offtargets[idx] += 1

    ## ------
    # CDS 2MM
    ## ------
    df_a7_2mm_cds = pd.read_csv(f'{species_name}/output/{track}/{species_name}_{track}_2mm_cds_alignment.sam', sep='\t',
                names=['query_name', 'strand', 'reference_name', 'mapping_position',
                       'aligned_seq', 'mapping_quality', 'idk', 'mismatch'])

    df_a7_2mm_cds.drop(columns=['idk', 'mapping_quality'], inplace=True)

    guide_w_pam = list()

    for idx, row in df_a7_2mm_cds.iterrows():
        if row['strand'] == '-':
            guide_w_pam.append(rc(row['aligned_seq']))
        else:
            guide_w_pam.append(row['aligned_seq'])

    df_a7_2mm_cds['guide_w_pam'] = guide_w_pam
    df_a7_2mm_cds['guide'] = df_a7_2mm_cds['guide_w_pam'].str[:-3]

    # Create a new column called mismatch_position and extract the mismatch position digits from
    # the mismatch column. Then fill NaN values with 0 because NaN means no mismatch.
    # Then drop rows where the mismatch occurs after the 10th base.
    df_a7_2mm_cds['mismatch'].fillna('Exact Match', inplace=True)
    df_a7_2mm_cds = df_a7_2mm_cds[~df_a7_2mm_cds['mismatch'].str.contains('20:')]
    df_a7_2mm_cds['num_mutations'] = df_a7_2mm_cds['mismatch'].apply(lambda x: len(x.split(',')))
    df_a7_2mm_cds.loc[df_a7_2mm_cds['mismatch'] == 'Exact Match', 'num_mutations'] = 0

    df_a7_2mm_cds['num_mutations'] = df_a7_2mm_cds['num_mutations'].astype(int)
    
    has_no_offtargets = (set(library['sequence']) == set(df_a7_2mm_cds['guide'])) and (len(library['sequence']) == len(df_a7_2mm_cds['guide']))

    if not has_no_offtargets:
        for _, row in df_a7_2mm_cds[df_a7_2mm_cds['num_mutations'] > 1].iterrows():
            idx = library[library['sequence'] == row['guide']].index[0]
            two_mm_cds_offtargets[idx] += 1
    
    for guide in set(df_a7_2mm_cds['guide']):
        df_a7_2mm_cds[df_a7_2mm_cds['guide'] == guide].to_csv(f'{species_name}/output/{track}/cds_targets/{guide}_cds_targets.csv', index=False)
    
    ## ----------
    ## Genome 0MM
    ## ----------
    df_a7_0mm_genomic = pd.read_csv(f'{species_name}/output/{track}/{species_name}_{track}_0mm_genomic_alignment.sam', sep='\t',
                    names=['query_name', 'strand', 'reference_name', 'mapping_position',
                        'aligned_seq', 'mapping_quality', 'idk', 'mismatch'])

    df_a7_0mm_genomic.drop(columns=['mapping_quality', 'idk'], inplace=True)

    guide_w_pam = list()

    for idx, row in df_a7_0mm_genomic.iterrows():
        if row['strand'] == '-':
            guide_w_pam.append(rc(row['aligned_seq']))
        else:
            guide_w_pam.append(row['aligned_seq'])

    df_a7_0mm_genomic['guide_w_pam'] = guide_w_pam
    df_a7_0mm_genomic['guide'] = df_a7_0mm_genomic['guide_w_pam'].str[:-3]

    # Create a new column called num_mutations and extract the mismatches from
    # the mismatch column.
    df_a7_0mm_genomic['mismatch'].fillna('Exact Match', inplace=True)
    df_a7_0mm_genomic = df_a7_0mm_genomic[~df_a7_0mm_genomic['mismatch'].str.contains('20:')]
    df_a7_0mm_genomic['num_mutations'] = df_a7_0mm_genomic['mismatch'].apply(lambda x: len(x.split(',')))
    df_a7_0mm_genomic.loc[df_a7_0mm_genomic['mismatch'] == 'Exact Match', 'num_mutations'] = 0

    df_a7_0mm_genomic['num_mutations'] = df_a7_0mm_genomic['num_mutations'].astype(int)
    
    has_no_offtargets = (set(library['sequence']) == set(df_a7_0mm_genomic['guide'])) and (len(library['sequence']) == len(df_a7_0mm_genomic['guide']))

    if not has_no_offtargets:
        for idx, row in library.iterrows():
            seq = row['sequence']
            zero_mm_genome_offtargets[idx] += len(df_a7_0mm_genomic[df_a7_0mm_genomic['guide'] == seq]) - 1

    ## ----------
    ## Genome 1MM
    ## ----------
    df_a7_1mm_genomic = pd.read_csv(f'{species_name}/output/{track}/{species_name}_{track}_1mm_genomic_alignment.sam', sep='\t',
                names=['query_name', 'strand', 'reference_name', 'mapping_position',
                       'aligned_seq', 'mapping_quality', 'idk', 'mismatch'])

    df_a7_1mm_genomic.drop(columns=['mapping_quality', 'idk'], inplace=True)

    guide_w_pam = list()

    for idx, row in df_a7_1mm_genomic.iterrows():
        if row['strand'] == '-':
            guide_w_pam.append(rc(row['aligned_seq']))
        else:
            guide_w_pam.append(row['aligned_seq'])

    df_a7_1mm_genomic['guide_w_pam'] = guide_w_pam
    df_a7_1mm_genomic['guide'] = df_a7_1mm_genomic['guide_w_pam'].str[:-3]

    # Create a new column called mismatch_position and extract the mismatch position digits from
    # the mismatch column. Then fill NaN values with 0 because NaN means no mismatch.
    # Then drop rows where the mismatch occurs after the 10th base.
    df_a7_1mm_genomic['mismatch'].fillna('Exact Match', inplace=True)
    df_a7_1mm_genomic = df_a7_1mm_genomic[~df_a7_1mm_genomic['mismatch'].str.contains('20:')]
    df_a7_1mm_genomic['num_mutations'] = df_a7_1mm_genomic['mismatch'].apply(lambda x: len(x.split(',')))
    df_a7_1mm_genomic.loc[df_a7_1mm_genomic['mismatch'] == 'Exact Match', 'num_mutations'] = 0

    df_a7_1mm_genomic['num_mutations'] = df_a7_1mm_genomic['num_mutations'].astype(int)
    
    has_no_offtargets = (set(library['sequence']) == set(df_a7_1mm_genomic['guide'])) and (len(library['sequence']) == len(df_a7_1mm_genomic['guide']))

    if not has_no_offtargets:
        for _, row in df_a7_1mm_genomic[df_a7_1mm_genomic['num_mutations'] > 0].iterrows():
            idx = library[library['sequence'] == row['guide']].index[0]
            one_mm_genome_offtargets[idx] += 1

    ## ----------
    ## Genome 2MM
    ## ----------
    df_a7_2mm_genomic = pd.read_csv(f'{species_name}/output/{track}/{species_name}_{track}_2mm_genomic_alignment.sam', sep='\t',
                names=['query_name', 'strand', 'reference_name', 'mapping_position',
                       'aligned_seq', 'mapping_quality', 'idk', 'mismatch'])

    df_a7_2mm_genomic.drop(columns=['mapping_quality', 'idk'], inplace=True)

    guide_w_pam = list()

    for idx, row in df_a7_2mm_genomic.iterrows():
        if row['strand'] == '-':
            guide_w_pam.append(rc(row['aligned_seq']))
        else:
            guide_w_pam.append(row['aligned_seq'])

    df_a7_2mm_genomic['guide_w_pam'] = guide_w_pam
    df_a7_2mm_genomic['guide'] = df_a7_2mm_genomic['guide_w_pam'].str[:-3]

    # Create a new column called num_mutations and extract the mismatches from
    # the mismatch column.
    df_a7_2mm_genomic['mismatch'].fillna('Exact Match', inplace=True)
    df_a7_2mm_genomic = df_a7_2mm_genomic[~df_a7_2mm_genomic['mismatch'].str.contains('20:')]
    df_a7_2mm_genomic['num_mutations'] = df_a7_2mm_genomic['mismatch'].apply(lambda x: len(x.split(',')))
    df_a7_2mm_genomic.loc[df_a7_2mm_genomic['mismatch'] == 'Exact Match', 'num_mutations'] = 0

    df_a7_2mm_genomic['num_mutations'] = df_a7_2mm_genomic['num_mutations'].astype(int)

    has_no_offtargets = (set(library['sequence']) == set(df_a7_2mm_genomic['guide'])) and (len(library['sequence']) == len(df_a7_2mm_genomic['guide']))

    if not has_no_offtargets:
        for _, row in df_a7_2mm_genomic[df_a7_2mm_genomic['num_mutations'] > 1].iterrows():
            idx = library[library['sequence'] == row['guide']].index[0]
            two_mm_genome_offtargets[idx] += 1

    for guide in set(df_a7_2mm_genomic['guide']):
        df_a7_2mm_genomic[df_a7_2mm_genomic['guide'] == guide].to_csv(f'{species_name}/output/{track}/genome_targets/{guide}_genome_targets.csv', index=False)
    
    zero_mm_cds_offtargets = np.array(zero_mm_cds_offtargets)
    one_mm_cds_offtargets = np.array(one_mm_cds_offtargets)
    two_mm_cds_offtargets = np.array(two_mm_cds_offtargets)

    zero_mm_genome_offtargets = np.array(zero_mm_genome_offtargets)
    one_mm_genome_offtargets = np.array(one_mm_genome_offtargets)
    two_mm_genome_offtargets = np.array(two_mm_genome_offtargets)

    library['0mm_cds_offtargets'] = zero_mm_cds_offtargets
    library['1mm_cds_offtargets'] = one_mm_cds_offtargets
    library['2mm_cds_offtargets'] = two_mm_cds_offtargets

    library['0mm_genome_offtargets'] = zero_mm_genome_offtargets
    library['1mm_genome_offtargets'] = one_mm_genome_offtargets
    library['2mm_genome_offtargets'] = two_mm_genome_offtargets

    library.to_csv(f'{species_name}/output/{track}/{track}_lib_offtargets.csv', index=False)

1. Align the augmented library to genome
    1. This finds the appropriate NGG PAM guide variants.
1. align augmented library to genome, then add a column without PAMS. This column must have the same elements and size as the unaugmented library guides. The size of the alignment will be:

1. Equal
    1. When either all guides have exactly 1 match (good!), or some guides have NO match (intron event) AND some have more than 1 match (offtarget event, double bad)
1. Fewer
    1. When some guides have no match (intron event)
1. More
    1. When some guides have more than 1 match (offtarget)

than the library size

In [10]:
species = ['yarrowia_lipolytica', 'kluyveromyces_marxianus',
           'komagataella_phaffii', 'saccharomyces_cerevisiae']

for track in ['a7', 'e1']:
    for s in species:
        GO(s, track)